In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import re
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from IPython.display import display, HTML


from aind_dynamic_foraging_data_utils import nwb_utils as nu
import aind_dynamic_foraging_basic_analysis.licks.annotation as a

from aind_dynamic_foraging_basic_analysis import plot_foraging_session, compute_foraging_efficiency

In [31]:
# Load single NWB

nwb = nu.load_nwb_from_filename('/root/capsule/data/foraging_nwb_bonsai/687582_2023-11-16_12-13-24.nwb')

/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [32]:
nwb

,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.004837,2.0,1.0,7.0,1.092118,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
1,4.489585e+06,4.489594e+06,2.0,False,False,4.489587e+06,4.489588e+06,4.489591e+06,True,False,0.8,0.711111,0.088889,0.526161,0.362175,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,1.675733,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
2,4.489594e+06,4.489605e+06,2.0,False,False,4.489597e+06,4.489599e+06,4.489602e+06,True,False,0.8,0.711111,0.088889,0.904736,0.315749,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,3.099859,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
3,4.489605e+06,4.489616e+06,2.0,False,False,4.489609e+06,4.489610e+06,4.489613e+06,True,False,0.8,0.711111,0.088889,0.758493,0.959065,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.024207,2.0,1.0,7.0,3.777559,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,2

In [33]:
# Create df_trial

nwb.df_trials = nu.create_df_trials(nwb)

nwb.df_trials

Timestamps are adjusted such that `_in_session` timestamps start at the first go cue


/opt/conda/lib/python3.9/site-packages/aind_dynamic_foraging_data_utils/nwb_utils.py:457: UserWarning: Reward before choice time. This is likely due to manual rewards
  warnings.warn("Reward before choice time. This is likely due to manual rewards")


,trial,animal_response,rewarded_historyL,rewarded_historyR,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,...,goCue_start_time_in_trial,reward_outcome_time_in_session,reward_outcome_time_in_trial,goCue_start_time_raw,reward_time_in_session,reward_time_in_trial,choice_time_in_session,choice_time_in_trial,earned_reward,extra_reward
0,0,2.0,False,False,False,False,0.8,0.711111,0.088889,0.963379,...,0.0,2.744864,2.744864,4.489579e+06,NaN,NaN,NaN,NaN,False,False
1,1,2.0,False,False,True,False,0.8,0.711111,0.088889,0.526161,...,0.0,11.984864,3.013856,4.489588e+06,NaN,NaN,NaN,NaN,False,False
2,2,2.0,False,False,True,False,0.8,0.711111,0.088889,0.904736,...,0.0,22.649856,3.013856,4.489599e+06,NaN,NaN,NaN,NaN,False,False
3,3,2.0,False,False,True,False,0.8,0.711111,0.088889,0.758493,...,0.0,33.501856,3.015840,4.489610e+06,NaN,NaN,NaN,NaN,False,False
4,4,2.0,False,False,True,False,0.8,0.711111,0.088889,0.555699,...,0.0,45.584864,3.013856,4.489622e+06,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,537,2.0,False,False,True,False,0.8,0.711111,0.088889,0.608060,...,0.0,5359.988864,3.013792,4.494936e+06,NaN,NaN,NaN,NaN,False,False
538,538,2.0,False,False,True,False,0.8,0.711111,0.088889,0.030398,...,0.0,5371.121856,3.013792,4.494947e+06,NaN,NaN,NaN,NaN,False,False
539,539,2.0,False,False,True,False,0.8,0.711111,0.088889,0.726525,...,0.0,5382.621856,3.013760,4.494959e+06,NaN,NaN,NaN,NaN,False,False
540,540,2.0,False,False,True,True,0.8,0.711111,0.088889,0.587486,...,0.0,5391.454880,3.013792,4.494968e+06,5388.442368,0.00128,NaN,NaN,False,True


In [7]:
LEFT, RIGHT, IGNORE = 0, 1, 2

In [47]:
def session_metrics(nwb):
    """
    Compute all session level metrics

    Includes session level metadata as a temporary organizer
    Includes majority of metrics from process_nwbs.py

    New addition: chosen_probability - average difference between the chosen probability
    and non-chosen probability / the difference between the largest and smallest probability
    in the session
    """

    if not hasattr(nwb, "df_trials"):
        print("You need to compute dfs: nwb_utils.create_trials_df(nwb)")
        return

    df = nwb.df_trials.copy()

    # METADATA PLACEHOLDER
    session_start_time = nwb.session_start_time
    session_date = session_start_time.strftime("%Y-%m-%d")
    subject_id_from_meta = nwb.subject.subject_id


    nwb.df_trials['non_autowater_trial'] = False
    nwb.df_trials.loc[(nwb.df_trials.auto_waterL==0) & (nwb.df_trials.auto_waterR==0), 'non_autowater_trial'] = True
    nwb.df_trials['non_autowater_finished_trial'] = nwb.df_trials['non_autowater_trial'] & (nwb.df_trials['animal_response'] != IGNORE)
    nwb.df_trials['ignored_non_autowater'] = nwb.df_trials['non_autowater_trial'] & (nwb.df_trials['animal_response'] == IGNORE)
    nwb.df_trials['ignored_autowater'] = ~nwb.df_trials['non_autowater_trial'] & (nwb.df_trials['animal_response'] == IGNORE)



    nwb.df_trials['reward_non_autowater'] = False  # Only reward from non-autowater trials (by definition, animal must not have ignored)
    nwb.df_trials.loc[(nwb.df_trials.rewarded_historyL | nwb.df_trials.rewarded_historyR), 'reward_non_autowater'] = True

    # Calculate performance metrics
    n_total_trials = len(df)
    n_finished_trials = (df.animal_response != IGNORE).sum()

    # Actual foraging trials (autowater excluded)
    n_total_trials_non_autowater = df.non_autowater_trial.sum()
    n_finished_trials_non_autowater = df.non_autowater_finished_trial.sum()

    n_reward_trials_non_autowater = df.reward_non_autowater.sum()
    reward_rate_non_autowater_finished = (
        n_reward_trials_non_autowater / n_finished_trials_non_autowater
        if n_finished_trials_non_autowater > 0
        else np.nan
    )

    # Calculate foraging efficiency
    foraging_eff, foraging_eff_random_seed = compute_foraging_efficiency(
        baited="without bait" not in nwb.protocol.lower(),
        choice_history=df.animal_response.map({0: 0, 1: 1, 2: np.nan}).values,
        reward_history=df.rewarded_historyL | df.rewarded_historyR,
        p_reward=[
            df.reward_probabilityL.values,
            df.reward_probabilityR.values,
        ],
        random_number=[
            df.reward_random_number_left.values,
            df.reward_random_number_right.values,
        ],
        autowater_offered=(df.auto_waterL == 1) | (df.auto_waterR == 1),
    )

    # Override foraging_eff_random_seed for old bpod sessions
    if "bpod" in nwb.session_description:
        foraging_eff_random_seed = nwb.get_scratch("metadata")[
            "foraging_efficiency_with_actual_random_seed"
        ].values[0]

    finished_rate = (
        n_finished_trials_non_autowater / n_total_trials_non_autowater
        if n_total_trials_non_autowater > 0
        else np.nan
    )

    # New Metrics

    # Probability chosen calculation
    probability_chosen = []
    probability_not_chosen = []

    for _, row in df.iterrows():
        if row.animal_response == 2:
            probability_chosen.append(np.nan)
            probability_not_chosen.append(np.nan)
        elif (
            row.animal_response == 0
        ):  # Chosen = left choice left probability, not chosen = left choice right probability
            probability_chosen.append(row.reward_probabilityL)
            probability_not_chosen.append(row.reward_probabilityR)
        else:  # Chosen = right choice right probability, not chosen = right choice left probability
            probability_chosen.append(row.reward_probabilityR)
            probability_not_chosen.append(row.reward_probabilityL)

    df["probability_chosen"] = probability_chosen
    df["probability_not_chosen"] = probability_not_chosen

    # Calculate the chosen probability
    average = df["probability_chosen"] - df["probability_not_chosen"]

    p_larger_global = max(
        df["probability_chosen"].max(), df["probability_not_chosen"].max()
    )

    p_smaller_global = min(
        df["probability_chosen"].min(), df["probability_not_chosen"].min()
    )

    mean_difference = average.mean()
    chosen_probability = mean_difference / (p_larger_global - p_smaller_global)

    # Pack all data
    dict_meta = {
        # Basic metadata
        "rig": meta_dict["box"],
        "user_name": nwb.experimenter[0],
        "task": nwb.protocol,
        # New metric
        "chosen_probability": chosen_probability,
        # Trial counts and rates
        "total_trials": n_total_trials_non_autowater,
        "finished_trials": n_finished_trials_non_autowater,
        "finished_rate": finished_rate,
        "total_trials_with_autowater": n_total_trials,
        "finished_trials_with_autowater": n_finished_trials,
        "finished_rate_with_autowater": n_finished_trials / n_total_trials,
        # Reward and foraging metrics
        "reward_trials": n_reward_trials_non_autowater,
        "reward_rate": reward_rate_non_autowater_finished,
        "foraging_eff": foraging_eff,
        "foraging_eff_random_seed": foraging_eff_random_seed,
        "foraging_performance": foraging_eff * finished_rate,
        "foraging_performance_random_seed": foraging_eff_random_seed * finished_rate,
        # Timing metrics
        "reaction_time_median": df.loc[:, "reaction_time"].median(),
        "reaction_time_mean": df.loc[:, "reaction_time"].mean(),
        "early_lick_rate": (df.loc[:, "n_lick_all_delay_period"] > 0).sum() / n_total_trials,
        # Double dipping metrics
        "double_dipping_rate_finished_trials": (
            df.loc[(df.animal_response != IGNORE), "n_lick_switches_gocue_stop"] > 0
        ).sum()
        / (df.animal_response != IGNORE).sum(),
        "double_dipping_rate_finished_reward_trials": (
            df.loc[df.reward, "n_lick_switches_gocue_stop"] > 0
        ).sum()
        / df.reward.sum(),
        "double_dipping_rate_finished_noreward_trials": (
            df.loc[
                (df.animal_response != IGNORE) & (~df.reward),
                "n_lick_switches_gocue_stop",
            ]
            > 0
        ).sum()
        / ((df.animal_response != IGNORE) & (~df.reward)).sum(),
        # Lick consistency metrics
        "lick_consistency_mean_finished_trials": df.loc[
            (df.animal_response != IGNORE), "n_lick_consistency_gocue_stop"
        ].mean(),
        "lick_consistency_mean_finished_reward_trials": df.loc[
            df.reward, "n_lick_consistency_gocue_stop"
        ].mean(),
        "lick_consistency_mean_finished_noreward_trials": df.loc[
            (df.animal_response != IGNORE) & (~df.reward),
            "n_lick_consistency_gocue_stop",
        ].mean(),
    }

    # Create DataFrame with hierarchical columns
    df_meta = pd.DataFrame(dict_meta)

    return df_meta

In [51]:
def session_metrics(nwb):
    """
    Compute all session level metrics

    Includes session level metadata as a temporary organizer
    Includes majority of metrics from process_nwbs.py

    New addition: chosen_probability - average difference between the chosen probability
    and non-chosen probability / the difference between the largest and smallest probability
    in the session
    """

    if not hasattr(nwb, "df_trials"):
        print("You need to compute dfs: nwb_utils.create_trials_df(nwb)")
        return

    df = nwb.df_trials.copy()

    # METADATA PLACEHOLDER
    session_start_time = nwb.session_start_time
    session_date = session_start_time.strftime("%Y-%m-%d")
    subject_id_from_meta = nwb.subject.subject_id


    # Calculate performance metrics
    n_total_trials = len(df)
    n_finished_trials = (df.animal_response != IGNORE).sum()

    # Actual foraging trials (autowater excluded)
    n_total_trials_non_autowater = df.non_autowater_trial.sum()
    n_finished_trials_non_autowater = df.non_autowater_finished_trial.sum()

    n_reward_trials_non_autowater = df.reward_non_autowater.sum()
    reward_rate_non_autowater_finished = (
        n_reward_trials_non_autowater / n_finished_trials_non_autowater
        if n_finished_trials_non_autowater > 0
        else np.nan
    )

    # Calculate foraging efficiency
    foraging_eff, foraging_eff_random_seed = compute_foraging_efficiency(
        baited="without bait" not in nwb.protocol.lower(),
        choice_history=df.animal_response.map({0: 0, 1: 1, 2: np.nan}).values,
        reward_history=df.rewarded_historyL | df.rewarded_historyR,
        p_reward=[
            df.reward_probabilityL.values,
            df.reward_probabilityR.values,
        ],
        random_number=[
            df.reward_random_number_left.values,
            df.reward_random_number_right.values,
        ],
        autowater_offered=(df.auto_waterL == 1) | (df.auto_waterR == 1),
    )

    # Override foraging_eff_random_seed for old bpod sessions
    if "bpod" in nwb.session_description:
        foraging_eff_random_seed = nwb.get_scratch("metadata")[
            "foraging_efficiency_with_actual_random_seed"
        ].values[0]

    finished_rate = (
        n_finished_trials_non_autowater / n_total_trials_non_autowater
        if n_total_trials_non_autowater > 0
        else np.nan
    )

    # New Metrics

    # Probability chosen calculation
    probability_chosen = []
    probability_not_chosen = []

    for _, row in df.iterrows():
        if row.animal_response == 2:
            probability_chosen.append(np.nan)
            probability_not_chosen.append(np.nan)
        elif (
            row.animal_response == 0
        ):  # Chosen = left choice left probability, not chosen = left choice right probability
            probability_chosen.append(row.reward_probabilityL)
            probability_not_chosen.append(row.reward_probabilityR)
        else:  # Chosen = right choice right probability, not chosen = right choice left probability
            probability_chosen.append(row.reward_probabilityR)
            probability_not_chosen.append(row.reward_probabilityL)

    df["probability_chosen"] = probability_chosen
    df["probability_not_chosen"] = probability_not_chosen

    # Calculate the chosen probability
    average = df["probability_chosen"] - df["probability_not_chosen"]

    p_larger_global = max(
        df["probability_chosen"].max(), df["probability_not_chosen"].max()
    )

    p_smaller_global = min(
        df["probability_chosen"].min(), df["probability_not_chosen"].min()
    )

    mean_difference = average.mean()
    chosen_probability = mean_difference / (p_larger_global - p_smaller_global)

    # Pack all data
    dict_meta = {
        # Basic metadata
        "user_name": nwb.experimenter[0],
        "task": nwb.protocol,
        # New metric
        "chosen_probability": chosen_probability,
        # Trial counts and rates
        "total_trials": n_total_trials_non_autowater,
        "finished_trials": n_finished_trials_non_autowater,
        "finished_rate": finished_rate,
        "total_trials_with_autowater": n_total_trials,
        "finished_trials_with_autowater": n_finished_trials,
        "finished_rate_with_autowater": n_finished_trials / n_total_trials,
        # Reward and foraging metrics
        "reward_trials": n_reward_trials_non_autowater,
        "reward_rate": reward_rate_non_autowater_finished,
        "foraging_eff": foraging_eff,
        "foraging_eff_random_seed": foraging_eff_random_seed,
        "foraging_performance": foraging_eff * finished_rate,
        "foraging_performance_random_seed": foraging_eff_random_seed * finished_rate,
        # Timing metrics
        "reaction_time_median": df.loc[:, "reaction_time"].median(),
        "reaction_time_mean": df.loc[:, "reaction_time"].mean(),
        "early_lick_rate": (df.loc[:, "n_lick_all_delay_period"] > 0).sum() / n_total_trials,
        # Double dipping metrics
        "double_dipping_rate_finished_trials": (
            df.loc[(df.animal_response != IGNORE), "n_lick_switches_gocue_stop"] > 0
        ).sum()
        / (df.animal_response != IGNORE).sum(),
        "double_dipping_rate_finished_reward_trials": (
            df.loc[df.reward, "n_lick_switches_gocue_stop"] > 0
        ).sum()
        / df.reward.sum(),
        "double_dipping_rate_finished_noreward_trials": (
            df.loc[
                (df.animal_response != IGNORE) & (~df.reward),
                "n_lick_switches_gocue_stop",
            ]
            > 0
        ).sum()
        / ((df.animal_response != IGNORE) & (~df.reward)).sum(),
        # Lick consistency metrics
        "lick_consistency_mean_finished_trials": df.loc[
            (df.animal_response != IGNORE), "n_lick_consistency_gocue_stop"
        ].mean(),
        "lick_consistency_mean_finished_reward_trials": df.loc[
            df.reward, "n_lick_consistency_gocue_stop"
        ].mean(),
        "lick_consistency_mean_finished_noreward_trials": df.loc[
            (df.animal_response != IGNORE) & (~df.reward),
            "n_lick_consistency_gocue_stop",
        ].mean(),
    }

    # Create DataFrame with hierarchical columns
    df_meta = pd.DataFrame(dict_meta)

    return df_meta

In [52]:
final_df = session_metrics(nwb)

KeyError: 'reaction_time'

In [ ]:
display(HTML(final_df.to_html()))